# 4. Junção de tabelas

In [2]:
import numpy as np
import pandas as pd

Vimos aqui como criar DFs, como filtrá-los, adicionar valores e agregar em novos DFs. O último ponto para manipulação de DFs é como será a interação entre dois diferentes DataFrames.

Há 3 principais modos de integração de dataframes:
    - **.merge()** - muito utilizado, se assemelha ao procv/procx/etc. do excel;
    - **.concat()** - parecido com concatenar strings só que em dataframes/tabelas, só adiciona do lado ou no final
    - **.join()** - confesso que nunca uso, tipo um .merge() com os parâmetros left_index e right_index = True, assim como o .groupby() está para o .pivot_table(), prefiro simplesmente ignorar a existência dessa função por ser um .merge() com menos opções

In [34]:
df_siglas = pd.read_json("Paises/names.json", orient="index").reset_index().rename(columns={"index": "Sigla", 0: "Nome"})
df_continentes = pd.read_json("Paises/continent.json", orient="index").reset_index().rename(columns={"index": "Sigla", 0: "Continente"})

In [25]:
df_times = pd.DataFrame(
    {
        "Mundiais": [2, np.nan, 2, 3, np.nan],
        "Libertadores": [1, 3, 3, 3, np.nan],
        "Fundação": [1910, 1914, 1912, 1930, 1913]
    },
    index=["Corinthians", "Palmeiras", "Santos", "São Paulo", "XV de Piracicaba"]
)

df_times_plus = pd.DataFrame(
    {
        "Mundiais": [np.nan, np.nan, np.nan, 2],
        "Libertadores": [np.nan, 2, 1, 1],
        "Fundação": [1902, 1921, 1898, 1910]
    },
    index=["Fluminense", "Cruzeiro", "Vasco", "Corinthians"]
)

df_times_2 = pd.DataFrame(
    {
        "Estádio": ["Arena Neo Química", "Allianz Park", "Vila Belmiro", "Barão da Serra Negra", "Maracanã"],
        "Cidade": ["São Paulo", "São Paulo", "Santos", "Piracicaba", "Maracanã"],
        "Mundiais": [2, np.nan, 2, np.nan, 1],
    },
    index=["Corinthians", "Palmeiras", "Santos", "XV de Piracicaba", "Flamengo"]
)



df_times

,Mundiais,Libertadores,Fundação
Corinthians,2.0,1.0,1910
Palmeiras,NaN,3.0,1914
Santos,2.0,3.0,1912
São Paulo,3.0,3.0,1930
XV de Piracicaba,NaN,NaN,1913


In [26]:
df_times_plus

,Mundiais,Libertadores,Fundação
Fluminense,NaN,NaN,1902
Cruzeiro,NaN,2.0,1921
Vasco,NaN,1.0,1898
Corinthians,2.0,1.0,1910


In [27]:
df_times_2

,Estádio,Cidade,Mundiais
Corinthians,Arena Neo Química,São Paulo,2.0
Palmeiras,Allianz Park,São Paulo,NaN
Santos,Vila Belmiro,Santos,2.0
XV de Piracicaba,Barão da Serra Negra,Piracicaba,NaN
Flamengo,Maracanã,Maracanã,1.0


In [28]:
df_times_plus

,Mundiais,Libertadores,Fundação
Fluminense,NaN,NaN,1902
Cruzeiro,NaN,2.0,1921
Vasco,NaN,1.0,1898
Corinthians,2.0,1.0,1910


## 4.1. .concat()

Seguiremos primeiramente com a função .concat() pois é a mais simples

In [29]:
pd.concat([df_times, df_times_plus]) # Perceba que os dois DataFrames foram unidos, concatenando as linhas

,Mundiais,Libertadores,Fundação
Corinthians,2.0,1.0,1910
Palmeiras,NaN,3.0,1914
Santos,2.0,3.0,1912
São Paulo,3.0,3.0,1930
XV de Piracicaba,NaN,NaN,1913
Fluminense,NaN,NaN,1902
Cruzeiro,NaN,2.0,1921
Vasco,NaN,1.0,1898
Corinthians,2.0,1.0,1910


In [30]:
pd.concat([df_times, df_times_plus], axis=1) # Perceba a concatenação por coluna

## ATENÇÃO
#  Tome muito cuidado para que não ocorra o que acontece abaixo, de ter mais de uma coluna com o msm nome ["Mundiais", "Libertadores", "Fundação", isso faria muito difícil de selecionar/filtrar uma coluna única (sem .iloc())

,Mundiais,Libertadores,Fundação,Mundiais,Libertadores,Fundação
Corinthians,2.0,1.0,1910.0,2.0,1.0,1910.0
Palmeiras,NaN,3.0,1914.0,NaN,NaN,NaN
Santos,2.0,3.0,1912.0,NaN,NaN,NaN
São Paulo,3.0,3.0,1930.0,NaN,NaN,NaN
XV de Piracicaba,NaN,NaN,1913.0,NaN,NaN,NaN
Fluminense,NaN,NaN,NaN,NaN,NaN,1902.0
Cruzeiro,NaN,NaN,NaN,NaN,2.0,1921.0
Vasco,NaN,NaN,NaN,NaN,1.0,1898.0


In [31]:
pd.concat([df_times, df_times_plus], axis=1, join="inner") # O join default é Outer (União), podemos selecionar Inner (Intersecção), caso necessário

,Mundiais,Libertadores,Fundação,Mundiais,Libertadores,Fundação
Corinthians,2.0,1.0,1910,2.0,1.0,1910


Há outros parâmetors no .concat(), porém muito provávelmente serão utilizados em casos específicos, como sempre segue a documentação para aprofundamento:
 <a href="https://pandas.pydata.org/docs/reference/api/pandas.concat.html?highlight=concat#pandas.concat">pandas.concat() </a>

# 4.2) .merge()

Partiremos agora para o .merge()

Algumas informações sobre o parâmetro how da .merge(), há 4 opções:
- inner (Default) - Pega somente a intersecção, ou seja, valores nos dois DataFrames
- outer - Pega a união, ou seja, os valores nos dois DataFrames, com valores nulos se não tem no DataFrame em questão
- left - Todos os valores do DataFrame da esquerda (o qual chama a função) são mantidos, tipo um procv()
- right - msm coisa com o DataFrame da direita, procv() inverso?

In [55]:

df_times.merge( # Dataframe da esquerda
    right=df_times_2, # Dataframe da direita
    left_index=True, # A chave do dataframe da esquerda será o index
    right_index=True, # A chave do dataframe da direita será o index
    how="outer", # how indicado a cima
    suffixes=["_base1", "_base2"] # Caso as colunas estejam duplicadas com valores diferentes, elas entram com esses sufixos

)

,Mundiais_base1,Libertadores,Fundação,Estádio,Cidade,Mundiais_base2
Corinthians,2.0,1.0,1910.0,Arena Neo Química,São Paulo,2.0
Flamengo,NaN,NaN,NaN,Maracanã,Maracanã,1.0
Palmeiras,NaN,3.0,1914.0,Allianz Park,São Paulo,NaN
Santos,2.0,3.0,1912.0,Vila Belmiro,Santos,2.0
São Paulo,3.0,3.0,1930.0,NaN,NaN,NaN
XV de Piracicaba,NaN,NaN,1913.0,Barão da Serra Negra,Piracicaba,NaN


In [36]:
df_siglas.head()

,Sigla,Nome
0,BD,Bangladesh
1,BE,Belgium
2,BF,Burkina Faso
3,BG,Bulgaria
4,BA,Bosnia and Herzegovina


In [42]:
df_continentes.head()

,Sigla,Continente
0,BD,AS
1,BE,EU
2,BF,AF
3,BG,EU
4,BA,EU


In [43]:
df_paises = df_siglas.copy()
df_paises = df_paises.merge(df_continentes, on="Sigla", how="left")
df_paises

,Sigla,Nome,Continente
0,BD,Bangladesh,AS
1,BE,Belgium,EU
2,BF,Burkina Faso,AF
3,BG,Bulgaria,EU
4,BA,Bosnia and Herzegovina,EU
...,...,...,...
245,IE,Ireland,EU
246,ID,Indonesia,AS
247,UA,Ukraine,EU
248,QA,Qatar,AS


## 4.3 .join()

Segue um exemplo utilizando .join(), veja que o resultado é o msm que usando .merge(). Bom saber caso você pegue um código que tenha .join() e saiba do que se trata, porém não recomendo a utilização.

In [56]:
df_times.join(df_times_2, lsuffix="_left", rsuffix="_right")

,Mundiais_left,Libertadores,Fundação,Estádio,Cidade,Mundiais_right
Corinthians,2.0,1.0,1910,Arena Neo Química,São Paulo,2.0
Palmeiras,NaN,3.0,1914,Allianz Park,São Paulo,NaN
Santos,2.0,3.0,1912,Vila Belmiro,Santos,2.0
São Paulo,3.0,3.0,1930,NaN,NaN,NaN
XV de Piracicaba,NaN,NaN,1913,Barão da Serra Negra,Piracicaba,NaN


In [57]:
df_times.merge(df_times_2, left_index=True, right_index=True, how="left", suffixes=["_left", "_right"])

,Mundiais_left,Libertadores,Fundação,Estádio,Cidade,Mundiais_right
Corinthians,2.0,1.0,1910,Arena Neo Química,São Paulo,2.0
Palmeiras,NaN,3.0,1914,Allianz Park,São Paulo,NaN
Santos,2.0,3.0,1912,Vila Belmiro,Santos,2.0
São Paulo,3.0,3.0,1930,NaN,NaN,NaN
XV de Piracicaba,NaN,NaN,1913,Barão da Serra Negra,Piracicaba,NaN
